# **Lab 3**

## **Task 1**

In [200]:
'''
For each case tabulate Re_c and Cd_o using numerical integration
'''

# Imports
import numpy as np
import pandas as pd

# Create dataframes
file_path = 'Lab_3_Data_Template_with_Data.xlsx'

df_50 = pd.read_excel(file_path, sheet_name="50 fps", usecols=[0,1,2,4], skiprows = 1)
df_100 = pd.read_excel(file_path, sheet_name="100 fps", usecols=[0,1,2,4,6], skiprows = 1)
df_150 = pd.read_excel(file_path, sheet_name="150 fps", usecols=[0,1,2,4], skiprows = 1)

for df in [df_50, df_100, df_150]:
    df["Actual Measurement (in)"] = [12.00, 11.75, 11.50, 11.25, 11.00, 10.75, 10.50, 10.25, 10.00, 9.75, 9.50, 9.38, 9.25, 9.13, 9.00, 8.88, 8.75, 8.63, 8.50, 8.25, 8.00, 7.75, 7.50, 7.25, 7.00]
    df["Relative Measurement (in)"] = df["Actual Measurement (in)"] - 9

# Constants
c = 6 * .0254 # Chord length in meters converted from inches
nu = 15.67e-6 # Kinematic viscosity in m2/s at 80F
g = 9.81 # Gravitational constant in m/s2

rho = [1.182, 1.182, 1.177] # Density in kg/m3 based on average temperature for each velocity

# Calculates Re
def calculate_re(u_atm):
    u_atm = u_atm * .3048 # Convert ft/s to m/s
    return u_atm * c / nu # Re

# Calculates downstream velocity (u2) based on recorded q
def calculate_u2(q, rho):
    q = q * 249.0889 # Convert inH20 to Pa
    return (q * 2 * rho) ** .5 # U2 in m/s

# Add Reynold's number to each dataframe
df_50["Re"] = calculate_re(50)
df_100["Re"] = calculate_re(100)
df_150["Re"] = calculate_re(150)

# Calculate Cd_o for each speed/aoa combo
i = 0
for df in [df_50, df_100, df_150]:
    u_atm = 50 * (1+i) * .3048
    ro = rho[i]
    q_atm = ro * u**2 / 2 

    df["U2 - 0"] = calculate_u2(df["Dynamic Pressure (in H2O)"], ro) # Downstream velocity for each location
    df["U2 - 8"] = calculate_u2(df["Dynamic Pressure (in H2O).1"], ro)
    try: df["U2 - 12"] = calculate_u2(df["Dynamic Pressure (in H2O).2"], ro)
    except: pass

    df["Cdo - 0 - Components"] = 1/q_atm * (ro * (u_atm**2 - df["U2 - 0"]**2) / g) # Each location's component of Cd_o
    df["Cdo - 8 - Components"] = 1/q_atm * (ro * (u_atm**2 - df["U2 - 8"]**2) / g)
    try: df["Cdo - 12 - Components"] = 1/q_atm * (ro * (u_atm**2 - df["U2 - 12"]**2) / g)
    except: pass

    df["Cdo - 0"] = df["Cdo - 0 - Components"].sum() # Total Cd_o for the AoA
    df["Cdo - 8"] = df["Cdo - 8 - Components"].sum()
    try: df["Cdo - 12"] = df["Cdo - 12 - Components"].sum()
    except: pass

    i += 1 

In [201]:
df_100

,Actual Measurement (in),Relative Measurement (in),Dynamic Pressure (in H2O),Dynamic Pressure (in H2O).1,Dynamic Pressure (in H2O).2,Re,U2 - 0,U2 - 8,U2 - 12,Cdo - 0 - Components,Cdo - 8 - Components,Cdo - 12 - Components,Cdo - 0,Cdo - 8,Cdo - 12
0,12.00,3.00,2.102,2.180,2.2180,296435.992342,35.181737,35.828545,36.139463,-0.030111,-0.034590,-0.036773,-0.69137,-0.673796,-0.009583
1,11.75,2.75,2.104,2.175,2.1990,296435.992342,35.198470,35.787434,35.984340,-0.030225,-0.034303,-0.035681,-0.69137,-0.673796,-0.009583
2,11.50,2.50,2.102,2.169,2.1840,296435.992342,35.181737,35.738037,35.861400,-0.030111,-0.033959,-0.034820,-0.69137,-0.673796,-0.009583
3,11.25,2.25,2.098,2.160,2.1370,296435.992342,35.148247,35.663815,35.473430,-0.029881,-0.033442,-0.032121,-0.69137,-0.673796,-0.009583
4,11.00,2.00,2.097,2.154,2.0370,296435.992342,35.139869,35.614248,34.633504,-0.029823,-0.033097,-0.026378,-0.69137,-0.673796,-0.009583
5,10.75,1.75,2.094,2.148,1.9020,296435.992342,35.114724,35.564611,33.466183,-0.029651,-0.032752,-0.018624,-0.69137,-0.673796,-0.009583
6,10.50,1.50,2.094,2.140,1.6700,296435.992342,35.114724,35.498321,31.358780,-0.029651,-0.032293,-0.005300,-0.69137,-0.673796,-0.009583
7,10.25,1.25,2.090,2.133,1.4000,296435.992342,35.081170,35.440215,28.712099,-0.029421,-0.031891,0.010206,-0.69137,-0.673796,-0.009583
8,10.00,1.00,2.088,2.127,1.1700,296435.992342,35.064381,35.390335,26.247857,-0.029307,-0.031546,0.023416,-0.69137,-0.673796,-0.009583
9,9.75,0.75,2.086,2.121,0.9980,296435.992342,35.047583,35.340383,24.241874,-0.029192,-0.031202,0.033294,-0.69137,-0.673796,-0.009583


$$ C_{d_o} = \frac{1}{\frac{1}{2} \rho {U_{atm}^2}} \Sigma \left[ \frac{\rho (U_{atm}^2 - U_{downstream}^2)}{g} \right]$$

In [202]:
# Refining data
index_values = [
    "50 FPS - 0 AoA",
    "50 FPS - 8 AoA",
    "100 FPS - 0 AoA",
    "100 FPS - 8 AoA",
    "100 FPS - 12 AoA",
    "150 FPS - 0 AoA",
    "150 FPS - 8 AoA"
]
refined_table = pd.DataFrame(index = index_values)
refined_table["Re"] = [df_50["Re"][0], df_50["Re"][0], df_100["Re"][0], df_100["Re"][0], df_100["Re"][0], df_150["Re"][0], df_150["Re"][0]]
refined_table["Cdo"] = [df_50["Cdo - 0"][0], df_50["Cdo - 8"][0], df_100["Cdo - 0"][0], df_100["Cdo - 8"][0], df_100["Cdo - 12"][0], df_150["Cdo - 0"][0], df_150["Cdo - 8"][0]]

In [203]:
refined_table

,Re,Cdo
50 FPS - 0 AoA,148217.996171,-0.140690
50 FPS - 8 AoA,148217.996171,-0.132129
100 FPS - 0 AoA,296435.992342,-0.691370
100 FPS - 8 AoA,296435.992342,-0.673796
100 FPS - 12 AoA,296435.992342,-0.009583
150 FPS - 0 AoA,444653.988513,-1.385487
150 FPS - 8 AoA,444653.988513,-0.933010


## **Task 2**

In [204]:
''' 
Calculate viscous drag by subtracting the form drag from the total drag. Calculate the
percentage of total drag for each component at each condition 
'''

form_drag = [-0.00457, 0.01646, 0.08246] # Form drag data provided by Dr. Cuppoletti at 100 ft/s and 0, 8, 12 AoA
total_drag = [refined_table["Cdo"][3], refined_table["Cdo"][4], refined_table["Cdo"][5]]

columns = pd.MultiIndex.from_tuples([("Value1", "Metric1"), ("Value1", "Metric2"), ("Value2", "Metric1"), ("Value2", "Metric2")], names=["Metric", "Type"])
viscous_drag = pd.DataFrame(index = index_values[2:5]) 
viscous_drag["Cdo"] = total_drag
viscous_drag["Form Drag"] = form_drag
viscous_drag["Viscous Drag"] = viscous_drag["Cdo"] - viscous_drag["Form Drag"]

In [205]:
viscous_drag

,Cdo,Form Drag,Viscous Drag
100 FPS - 0 AoA,-0.673796,-0.00457,-0.669226
100 FPS - 8 AoA,-0.009583,0.01646,-0.026043
100 FPS - 12 AoA,-1.385487,0.08246,-1.467947


In [206]:
columns = pd.MultiIndex.from_tuples([("50", "0 AoA"), ("50", "8 AoA"), ("100", "0 AoA"), ("100", "8 AoA"), ("100", "12 AoA"), ("150", "0 AoA"), ("150", "8 AoA")], names=["", "Position"])
drag_percentages = pd.DataFrame(index = [12.00, 11.75, 11.50, 11.25, 11.00, 10.75, 10.50, 10.25, 10.00, 9.75, 9.50, 9.38, 9.25, 9.13, 9.00, 8.88, 8.75, 8.63, 8.50, 8.25, 8.00, 7.75, 7.50, 7.25, 7.00], columns = columns)

In [216]:
drag_percentages.iloc[:, 0] = (df_50["Cdo - 0 - Components"].to_list()      / df_50["Cdo - 0 - Components"].sum() * 100)
drag_percentages.iloc[:, 1] = (df_50["Cdo - 8 - Components"].to_list()      / df_50["Cdo - 8 - Components"].sum() * 100)
drag_percentages.iloc[:, 2] = (df_100["Cdo - 0 - Components"].to_list()     / df_100["Cdo - 0 - Components"].sum() * 100)
drag_percentages.iloc[:, 3] = (df_100["Cdo - 8 - Components"].to_list()     / df_100["Cdo - 8 - Components"].sum() * 100)
drag_percentages.iloc[:, 4] = (df_100["Cdo - 12 - Components"].to_list()    / df_100["Cdo - 12 - Components"].sum() * 100)
drag_percentages.iloc[:, 5] = (df_150["Cdo - 0 - Components"].to_list()     / df_150["Cdo - 0 - Components"].sum() * 100)
drag_percentages.iloc[:, 6] = (df_150["Cdo - 8 - Components"].to_list()     / df_150["Cdo - 8 - Components"].sum() * 100)

# Format all values in the drag_percentages DataFrame as percentages with two decimal places
drag_percentages = drag_percentages.applymap('{:.2f}%'.format)


In [217]:
drag_percentages

50           100                      150        
Position  0 AoA  8 AoA  0 AoA   8 AoA    12 AoA  0 AoA   8 AoA
12.00     4.31%  4.84%  4.36%   5.13%   383.71%  4.43%   5.58%
11.75     4.31%  4.83%  4.37%   5.09%   372.32%  4.47%   5.55%
11.50     4.30%  4.80%  4.36%   5.04%   363.33%  4.45%   5.42%
11.25     4.27%  4.77%  4.32%   4.96%   335.17%  4.34%   5.31%
11.00     4.28%  4.73%  4.31%   4.91%   275.24%  4.43%   5.24%
10.75     4.24%  4.70%  4.29%   4.86%   194.34%  4.35%   5.14%
10.50     4.25%  4.68%  4.29%   4.79%    55.31%  4.31%   5.08%
10.25     4.23%  4.64%  4.26%   4.73%  -106.50%  4.28%   4.92%
10.00     4.23%  4.63%  4.24%   4.68%  -244.33%  4.26%   4.89%
9.75      4.17%  4.56%  4.22%   4.63%  -347.41%  4.25%   4.74%
9.50      4.15%  4.52%  4.21%   4.55%  -422.44%  4.26%   4.70%
9.38      4.15%  4.51%  4.21%   4.51%  -433.46%  4.25%   4.65%
9.25      4.04%  4.48%  4.20%   4.49%  -433.04%  4.22%   4.60%
9.13      3.80%  4.37%  3.75%   4.52%  -398.53%  4.04%   4.59%
9.00      3.41%  3.86%  2.68%   4.49%  -363.77%  2.49%   4.57%
8.88      3.00%  3.05%  2.07%   4.23%  -303.66%  1.33%   4.46%
8.75      2.94%  2.07%  2.84%   2.87%  -248.53%  2.53%   3.54%
8.63      3.30%  1.24%  3.72%   0.72%  -173.62%  3.87%   0.91%
8.50      3.83%  0.85%  4.12%  -0.92%   -69.94%  4.19%  -2.19%
8.25      4.12%  2.18%  4.21%   0.62%   100.85%  4.23%  -1.99%
8.00      4.11%  4.04%  4.16%   3.99%   238.68%  4.19%   3.51%
7.75      4.10%  4.38%  4.16%   4.29%   306.40%  4.16%   4.21%
7.50      4.14%  4.43%  4.20%   4.28%   335.77%  4.22%   4.17%
7.25      4.17%  4.43%  4.23%   4.26%   341.16%  4.22%   4.22%
7.00      4.15%  4.40%  4.25%   4.26%   342.96%  4.25%   4.15%